In [1]:
from tqdm import tqdm
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

/var/data/python/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df_anomaly = pd.read_csv('/home/thetom205/traktor hack/data/для анализа/dataset._anomaly.csv',sep=';')
df_anomaly.replace('        -', np.nan, inplace=True)
df_anomaly.replace('-', np.nan, inplace=True)

/tmp/ipykernel_795/1735381011.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_anomaly.replace('        -', np.nan, inplace=True)
/tmp/ipykernel_795/1735381011.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_anomaly.replace('-', np.nan, inplace=True)


In [3]:
df_problems = pd.read_csv('/home/thetom205/traktor hack/data/для анализа/dataset._problems.csv',sep=';')
df_problems.replace('        -', np.nan, inplace=True)
df_problems.replace('-', np.nan, inplace=True)

/tmp/ipykernel_795/3875790439.py:1: DtypeWarning: Columns (3,11,12,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_problems = pd.read_csv('/home/thetom205/traktor hack/data/для анализа/dataset._problems.csv',sep=';')
/tmp/ipykernel_795/3875790439.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_problems.replace('        -', np.nan, inplace=True)
/tmp/ipykernel_795/3875790439.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
 

In [4]:
df_normal = pd.read_csv('/home/thetom205/traktor hack/data/для анализа/dataset._normal.csv',sep=';')
df_normal.replace('        -', np.nan, inplace=True)
df_normal.replace('-', np.nan, inplace=True)

/tmp/ipykernel_795/4066241437.py:1: DtypeWarning: Columns (3,12,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_normal = pd.read_csv('/home/thetom205/traktor hack/data/для анализа/dataset._normal.csv',sep=';')
/tmp/ipykernel_795/4066241437.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_normal.replace('        -', np.nan, inplace=True)
/tmp/ipykernel_795/4066241437.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_norma

Метка 0 - нормальное состояние, 1 - аномалия, 2 - поломка

In [5]:
df_normal['target'] = 0
df_anomaly['target'] = 1
df_problems['target'] = 2

In [6]:
train = pd.concat([
    df_normal,
    df_anomaly,
    df_problems
])

In [7]:
column_names_train = train.columns.tolist()
for i in train:
  if (train[i].isnull().sum() / len(train[i])) * 100 > 90:
    del train[i]

In [8]:
train.dropna(thresh=5, inplace=True, axis=0)

In [9]:
for i in train.select_dtypes(include=['object']):
   train[i] = train[i].fillna(train[i].mode().iloc[0])

In [10]:
def make_harmonic_features(value, period=24):
    value *= 2 * np.pi / period
    return np.cos(value), np.sin(value)

In [11]:
train['Темп.масла двиг.,°С'] = train['Темп.масла двиг.,°С'].str.replace(',', '.').astype('float')
train['Обор.двиг.,об/мин'] = train['Обор.двиг.,об/мин'].str.replace(',', '.').astype('float')
train['Полож.пед.акселер.,%'] = train['Полож.пед.акселер.,%'].str.replace(',', '.').astype('float')

In [12]:
train_DVS = train[['Значение счетчика моточасов, час:мин','Давл.масла двиг.,кПа','Полож.пед.акселер.,%','Сост.пед.сцепл.','Темп.масла двиг.,°С','Обор.двиг.,об/мин','ДВС. Давление смазки','ДВС. Температура охлаждающей жидкости','ДВС. Частота вращения коленчатого вала','Аварийная температура охлаждающей жидкости (spn3841)','Аварийное давление масла ДВС (spn3846)','Засоренность фильтра ДВС (spn3845)','Аварийная температура масла ДВС(spn3856)','Низкий уровень ОЖ (spn3860)','Подогрев топливного фильтра (spn3865)','Скорость','Давление в пневмостистеме (spn46), кПа','Засоренность воздушного фильтра (spn3840)','Недопустимый уровень масла в гидробаке (spn3850)','Аварийная температура масла в гидросистеме (spn3849)','Выход блока управления двигателем (spn3852)','Дата и время', 'target']]
train_DVS[['Сост.пед.сцепл.','Значение счетчика моточасов, час:мин']] = train_DVS[['Сост.пед.сцепл.','Значение счетчика моточасов, час:мин']].astype('str')
train_DVS = train_DVS.drop(['Дата и время'], axis=1)
train_DVS[['Давл.масла двиг.,кПа','ДВС. Давление смазки','ДВС. Температура охлаждающей жидкости']] = train_DVS[['Давл.масла двиг.,кПа','ДВС. Давление смазки','ДВС. Температура охлаждающей жидкости']].astype('float64')

/tmp/ipykernel_795/2622125198.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_DVS[['Сост.пед.сцепл.','Значение счетчика моточасов, час:мин']] = train_DVS[['Сост.пед.сцепл.','Значение счетчика моточасов, час:мин']].astype('str')


In [13]:
cat = ['Значение счетчика моточасов, час:мин', 'Сост.пед.сцепл.']

In [14]:
X, y = train_DVS.drop(['target'], axis=1), train_DVS.target

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

In [16]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, stratify=y_test, test_size=0.3, random_state=42)

In [17]:
# model = CatBoostClassifier(
#     iterations=200, depth=10, learning_rate=0.1,
#    loss_function='MultiClass', verbose=100)
# model.fit(X_train, y_train,eval_set=(X_val,y_val),plot=True)

In [19]:
from joblib import load

model = load(r"model_dvs.pkl")

In [20]:
y_pred = model.predict(X_test)

In [21]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.9697954194544519
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.87      0.91     98995
           1       1.00      1.00      1.00     39838
           2       0.97      0.99      0.98    451839

    accuracy                           0.97    590672
   macro avg       0.97      0.95      0.96    590672
weighted avg       0.97      0.97      0.97    590672



In [22]:
y_pred_val = model.predict(X_val)

accuracy = accuracy_score(y_val, y_pred_val)
report = classification_report(y_val, y_pred_val)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.9700765566115996
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.87      0.91     42427
           1       1.00      1.00      1.00     17073
           2       0.97      0.99      0.98    193646

    accuracy                           0.97    253146
   macro avg       0.97      0.95      0.96    253146
weighted avg       0.97      0.97      0.97    253146



In [23]:
df_imp = pd.DataFrame({
    'name': X.columns,
    'imp': model.get_feature_importance()
}).sort_values('imp', ascending=False)

In [ ]:
df_imp

In [24]:
from joblib import dump

dump(model, "model_dvs.pkl")

['model_dvs.pkl']

In [25]:
y_proba_val = model.predict_proba(X_val)

In [26]:
df_fin = pd.DataFrame({
    'normal':y_proba_val[:, 0],
    'anomaly': y_proba_val[:, 1],
    'problem' : y_proba_val[:, 2]
})

In [27]:
df_fin['normal'] = df_fin['normal'].mean()
df_fin['anomaly'] = df_fin['anomaly'].mean()
df_fin['problem'] = df_fin['problem'].mean()

In [28]:
df_fin.drop_duplicates()

,normal,anomaly,problem
0,0.175838,0.080462,0.7437
